<a href="https://colab.research.google.com/github/wincmoriya/pytorch-handson/blob/main/ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ResNet実装
* 学習自体は行わない
* 残差ブロックを何層も積み重ねてResNetを作る

In [1]:
import torch
import torch.nn as nn

## 残差ブロックの定義
* 残差ブロックではwight_layerが2つあるので積み上げも2つ必要
* スキップコネクションである恒等関数も合流させる

In [3]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        # いつものようにnn.Sequentialでネットワーク構築するのではない
        # スキップコネクション等複雑な操作がでてくるので使うパーツを定義してからdef forwardの中でネットワーク構築する

        # 1つ目の畳み込み層を定義, kernel_size, paddingは一般的な値にする
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        # 2つ目、out_channelsを受け取ってout_channelsを出力する畳み込み層にする
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
        # 3つ目、kernel_size=1 -> あるチャネル数を持ったものを別のチャネル数に変化させる時に、このようなオプションにする
        self.conv3 = nn.Conv2d(in_channels, out_channels, kernel_size=1, padding=0)
        # バッチノーマライゼーションの定義、引数にはチャネル数が必要
        self.bn = nn.BatchNorm2d(out_channels)
        # ReLUを渡す
        self.relu = nn.ReLU()
    def shortcut(self, x):
        x = self.conv3(x)
        x = self.bn(x)
        return x
    def forward(self, x):
        # 恒等関数の設定
        identity = x
        # 最初の畳み込みを行う
        x = self.conv1(x)
        # バッチノーマライゼーションを行う
        x = self.bn(x)
        x = self.relu(x)
        # 2回目の畳み込みを行う
        x = self.conv2(x)
        x = self.bn(x)
        # 恒等関数を足す
        # x += identityという風に普通に恒等関数を足すとエラーが起きる
        # identityに入っているのは入力のin_channelだが最後のxはout_channelが入っていてチャネル数が違うのでエラーが起きてしまう
        # そこでチャネル数を合わせるshortcut()で変換する
        x += self.shortcut(identity)
        return x


## 残差ブロックを何層も積み重ねてResNetを作成する

In [5]:
class ResNet(nn.Module):
    # 残差ブロックを渡すための変数block
    def __init__(self, block):
        super().__init__()
        # 例) 28*28*64で28*28の画像*out_channelsが64, out_featuresは10クラス分類を想定する
        self.linear = nn.Linear(in_features=28*28*64, out_features=10)
        # _make_layer(block, 3層積み重ね, カラー画像想定で3, 出力は64)
        self.layer = self._make_layer(block, 3, 3, 64)
    # _make_layer(self, ブロック, 何層作るか, 入力, 出力)
    def _make_layer(self, block, num_residual_blocks, in_channels, out_channels):
        # 空リストに何層も積み上げていくイメージ
        layers = []
        # 積み上げたい数だけforループを回す
        for i in range(num_residual_blocks):
            # 最初のブロックだけは入力画像見たいな入力をするので、in_channelsが3とかっていう数値になる、out_channelsは自分が指定した値
            if i==0:
                layers.append(block(in_channels, out_channels))
            # 2層目以降はout_channelsをin_channelsの部分で渡す
            else:
                layers.append(block(out_channels, out_channels))
        # 複数のブロックを積み重ねたモデルを渡す
        return nn.Sequential(*layers)
    def forward(self, x):
        # 3層のresidualブロックの順伝播が行われる
        x = self.layer(x)
        # xはチャネル数が64の画像が出てきて、このままだとリニア層(全結合層)に代入できないのでviewを使う
        # view(バッチ数の次元, c*h*wの一次元のベクトルに変換)
        x = x.view(x.size(0), -1)
        x = self.linear(x)
        return x

In [6]:
model = ResNet(ResidualBlock)

In [7]:
# 3層のresidual_block(0, 1, 2)が作成されていることがわかる
model

ResNet(
  (linear): Linear(in_features=50176, out_features=10, bias=True)
  (layer): Sequential(
    (0): ResidualBlock(
      (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv3): Conv2d(3, 64, kernel_size=(1, 1), stride=(1, 1))
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
    (1): ResidualBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv3): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
    (2): ResidualBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1,

In [8]:
# テストの配列を使って順伝播を計算してみる
# randn(ミニバッチサイズ32、チャネル数3、画像サイズ28*28)
x_test = torch.randn(32, 3, 28, 28)

In [9]:
output = model(x_test)

In [11]:
# ミニバッチ数32で10個の分類のものができる
output.size()

torch.Size([32, 10])